In [36]:
#Script to reformat flux xls file for use in a stacked bar chart

#Created: 12.04.2016
#Last modified: 14.04.2016 Sorted data by abs_anthro value; added abs_anthro_perArea col
#               18.04.2016 Output carbon flux per surface area instead of unit area

In [1]:
import numpy as np
import pandas as pd
import collections
import os
import xlrd

In [45]:
molarmassC = 12.011 #g/mol

In [141]:
# Create dataframe from flux file

xl = pd.ExcelFile("/homel/cnangini/PROJECTS/CO2airsea/for_DataViz_complete.xls")
xl.sheet_names
#[u'Sheet1', u'Sheet2', u'Sheet3']
#[u'Sheet1']
df = xl.parse("Sheet1")
df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,ORCA2,Unnamed: 5,Unnamed: 6,ORCA05,Unnamed: 8,Unnamed: 9,ORCA025,Unnamed: 11,Unnamed: 12
0,NaN,"Laruelle et al. (2013, 2014)",NaN,NaN,Surface\n(10^3 km^2),Air-sea flux\n(mol C m^-2 yr^-1),NaN,Surface\n(10^3 km^2),Air-sea flux\n(mol C m^-2 yr^-1),NaN,Surface\n(10^3 km^2),Air-sea flux\n(mol C m^-2 yr^-1),NaN
1,ID,MARCATS,CLASS,Mean Latitude (°N),NaN,Total,Anthropogenic,NaN,Total,Anthropogenic,NaN,Total,Anthropogenic
2,2,California Current,EBC,31.43,71.9735,0.258775,-0.192213,118.311,-0.33556,-0.35,197.109,-0.408655,-0.3447023
3,4,Peruvian Upwelling Current,EBC,-20.98,130.317,-0.0424662,-0.4052452,137.516,1.44461,-0.39,143.984,1.40228,-0.34623
4,19,Iberian Upwelling,EBC,47.39,174.844,-0.953769,-0.298491,251.093,-1.12506,-0.27,278.965,-0.850057,-0.235254
5,22,Moroccan Upwelling,EBC,22.46,111.796,0.192162,-0.211935,176.947,0.18122,-0.33,210.894,0.201397,-0.343572
6,24,Southern Western Africa,EBC,-27.99,307.138,-1.14521,-0.532412,299.792,0.428844,-0.59,306.152,0.413103,-0.578043
7,33,Leeuwin Current,EBC,-28.21,38.0426,-0.909239,-0.257277,81.4649,-2.05471,-0.6,110.191,-1.8733,-0.56869
8,27,Western Arabian Sea,Indian margins,17.08,20.4899,-0.062801,-0.200482,81.8175,0.347792,-0.31,104.031,0.470945,-0.309286
9,30,Eastern Arabian Sea,Indian margins,16.61,340.797,0.1413,-0.32113,323.5,0.209194,-0.3,355.181,0.285837,-0.254938


In [142]:
#model ORCA05 only
df05 = df.drop(['Unnamed: 0', 'ORCA2', 'Unnamed: 5', 'Unnamed: 6', 'ORCA025', 'Unnamed: 11', 'Unnamed: 12'], 1)
df05.tail(5)

,Unnamed: 1,Unnamed: 2,Unnamed: 3,ORCA05,Unnamed: 8,Unnamed: 9
42,Brazilian Current,WBC,-27.06,474.457,-0.328442,-0.34
43,Florida Upwelling,WBC,40.62,784.725,-2.08861,-0.56
44,Agulhas Current,WBC,-28.37,189.285,-1.20098,-0.53
45,Eastern Austrialian Current,WBC,-38.11,97.4527,-1.73964,-0.5
46,China Sea & Kuroshio,WBC,33.13,1131.67,-1.9898,-0.45


In [143]:
#rename columns
df05.columns = ['MARCATS', 'RCLASS', 'LAT', 'AREA', 'TOTAL', 'ANTHRO']
df05.head(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO
0,"Laruelle et al. (2013, 2014)",NaN,NaN,Surface\n(10^3 km^2),Air-sea flux\n(mol C m^-2 yr^-1),NaN
1,MARCATS,CLASS,Mean Latitude (°N),NaN,Total,Anthropogenic
2,California Current,EBC,31.43,118.311,-0.33556,-0.35
3,Peruvian Upwelling Current,EBC,-20.98,137.516,1.44461,-0.39
4,Iberian Upwelling,EBC,47.39,251.093,-1.12506,-0.27


In [144]:
#Remove rows 0 and 1
df05 = df05.drop([0, 1], 0)
df05.head(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO
2,California Current,EBC,31.43,118.311,-0.33556,-0.35
3,Peruvian Upwelling Current,EBC,-20.98,137.516,1.44461,-0.39
4,Iberian Upwelling,EBC,47.39,251.093,-1.12506,-0.27
5,Moroccan Upwelling,EBC,22.46,176.947,0.18122,-0.33
6,Southern Western Africa,EBC,-27.99,299.792,0.428844,-0.59


In [145]:
#Add column for abs value of ANTRHO for plotting purposes
df05['abs_anthro'] = -1 * df05['ANTHRO']
df05.tail(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro
42,Brazilian Current,WBC,-27.06,474.457,-0.328442,-0.34,0.34
43,Florida Upwelling,WBC,40.62,784.725,-2.08861,-0.56,0.56
44,Agulhas Current,WBC,-28.37,189.285,-1.20098,-0.53,0.53
45,Eastern Austrialian Current,WBC,-38.11,97.4527,-1.73964,-0.5,0.5
46,China Sea & Kuroshio,WBC,33.13,1131.67,-1.9898,-0.45,0.45


In [146]:
#Add columns for flux per surface area, expressed in Tg (10e12 g)
#Convert fluxes to Tg by multiplying by molarmassC and 1e-12, and convert AREA to m2 by multiplying by 1e9

df05['total_perArea'] = df05['TOTAL'] * molarmassC * df05['AREA'] * 1e9 * 1e-12
df05['anthro_perArea'] = df05['ANTHRO'] * molarmassC * df05['AREA'] * 1e9 * 1e-12
df05['abs_anthro_perArea'] = df05['abs_anthro'] * molarmassC * df05['AREA'] * 1e9 * 1e-12
df05.head(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro,total_perArea,anthro_perArea,abs_anthro_perArea
2,California Current,EBC,31.43,118.311,-0.33556,-0.35,0.35,-0.476842,-0.4973617,0.4973617
3,Peruvian Upwelling Current,EBC,-20.98,137.516,1.44461,-0.39,0.39,2.386069,-0.6441648,0.6441648
4,Iberian Upwelling,EBC,47.39,251.093,-1.12506,-0.27,0.27,-3.393044,-0.8142871,0.8142871
5,Moroccan Upwelling,EBC,22.46,176.947,0.18122,-0.33,0.33,0.3851488,-0.7013524,0.7013524
6,Southern Western Africa,EBC,-27.99,299.792,0.428844,-0.59,0.59,1.544182,-2.124473,2.124473


In [147]:
df05

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro,total_perArea,anthro_perArea,abs_anthro_perArea
2,California Current,EBC,31.43,118.311,-0.33556,-0.35,0.35,-0.476842,-0.4973617,0.4973617
3,Peruvian Upwelling Current,EBC,-20.98,137.516,1.44461,-0.39,0.39,2.386069,-0.6441648,0.6441648
4,Iberian Upwelling,EBC,47.39,251.093,-1.12506,-0.27,0.27,-3.393044,-0.8142871,0.8142871
5,Moroccan Upwelling,EBC,22.46,176.947,0.18122,-0.33,0.33,0.3851488,-0.7013524,0.7013524
6,Southern Western Africa,EBC,-27.99,299.792,0.428844,-0.59,0.59,1.544182,-2.124473,2.124473
7,Leeuwin Current,EBC,-28.21,81.4649,-2.05471,-0.6,0.6,-2.010482,-0.5870849,0.5870849
8,Western Arabian Sea,Indian margins,17.08,81.8175,0.347792,-0.31,0.31,0.3417787,-0.3046401,0.3046401
9,Eastern Arabian Sea,Indian margins,16.61,323.5,0.209194,-0.3,0.3,0.8128355,-1.165668,1.165668
10,Bay of Bengal,Indian margins,18.3,197.021,-0.693303,-0.31,0.31,-1.640646,-0.73359,0.73359
11,Tropical Eastern Indian,Indian margins,-3.274,726.646,-0.0552213,-0.2,0.2,-0.4819574,-1.745549,1.745549


In [148]:
#Sort by abs_anthro_perArea
#df_stats.sort('MtCO2', ascending=False).head(50)
df05 = df05.sort('abs_anthro_perArea', ascending=False)
df05.head(10)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro,total_perArea,anthro_perArea,abs_anthro_perArea
28,Antarctic Shelves,Polar,-71.76,2462,-0.904177,-0.69,0.69,-26.73749,-20.40405,20.40405
14,North Eastern Atlantic,Marginal sea,56.69,1020.02,-2.16314,-0.53,0.53,-26.50162,-6.493274,6.493274
40,Northern Australia,Tropical,-11.15,2278.28,-0.287668,-0.23,0.23,-7.871868,-6.293817,6.293817
30,Southern America,Subpolar,-47.92,1126.28,-1.5125,-0.46,0.46,-20.46072,-6.222765,6.222765
46,China Sea & Kuroshio,WBC,33.13,1131.67,-1.9898,-0.45,0.45,-27.04633,-6.11662,6.11662
43,Florida Upwelling,WBC,40.62,784.725,-2.08861,-0.56,0.56,-19.68584,-5.278186,5.278186
41,South East Asia,Tropical,8.667,2129.62,-0.287109,-0.2,0.2,-7.343923,-5.115773,5.115773
21,Sea of Okhotsk,Marginal sea,55.61,932.861,-1.66217,-0.36,0.36,-18.62394,-4.033654,4.033654
34,North Western Pacific,Subpolar,60.17,1024.93,-1.84886,-0.24,0.24,-22.76027,-2.954504,2.954504
24,Southern Greenland,Polar,64.75,237.64,-3.35089,-0.86,0.86,-9.564425,-2.454693,2.454693


In [149]:
df05.tail(10)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro,total_perArea,anthro_perArea,abs_anthro_perArea
7,Leeuwin Current,EBC,-28.21,81.4649,-2.05471,-0.6,0.6,-2.010482,-0.5870849,0.5870849
45,Eastern Austrialian Current,WBC,-38.11,97.4527,-1.73964,-0.5,0.5,-2.036256,-0.5852522,0.5852522
38,Tropical Eastern Atlantic,Tropical,2.095,225.243,0.0884754,-0.19,0.19,0.2393608,-0.5140248,0.5140248
2,California Current,EBC,31.43,118.311,-0.33556,-0.35,0.35,-0.476842,-0.4973617,0.4973617
8,Western Arabian Sea,Indian margins,17.08,81.8175,0.347792,-0.31,0.31,0.3417787,-0.3046401,0.3046401
19,Persian Gulf,Marginal sea,26.9,207.628,0.0369855,-0.12,0.12,0.09223518,-0.2992584,0.2992584
18,Red Sea,Marginal sea,18.31,157.725,0.242964,-0.15,0.15,0.4602795,-0.2841652,0.2841652
17,Black Sea,Marginal sea,44.75,130.532,-0.239324,-0.18,0.18,-0.3752169,-0.2822076,0.2822076
39,Tropical Western Indian,Tropical,-13.9,46.4441,-0.0559819,-0.16,0.16,-0.03122895,-0.08925441,0.08925441
15,Baltic Sea,Marginal sea,58.98,324.209,0.303937,-0.01,0.01,1.183553,-0.03894074,0.03894074


In [150]:
#============================
# PART I - STACKED BAR CHART
# Build tsv file
#============================

#Pivot table so that MARCATS col becomes first row with values corresponding to abs_anthro
#http://stackoverflow.com/questions/29942167/transposing-one-column-in-python-pandas-with-the-simplest-index-possible
#dc.pivot(columns='disease_header', index='medicine', values='disease').reset_index()

df05_piv = df05.pivot(columns='MARCATS', index='RCLASS', values='abs_anthro').reset_index()
#df05_piv = df05_piv.drop(['CLASS'], 1)
df05_piv

MARCATS,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.35,NaN,...,NaN,NaN,NaN,0.59,NaN,NaN,NaN,NaN,NaN,NaN
1,Indian margins,NaN,NaN,NaN,NaN,0.31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,0.31
2,Marginal sea,NaN,NaN,0.01,NaN,NaN,0.18,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Polar,NaN,0.69,NaN,0.11,NaN,NaN,NaN,NaN,0.09,...,NaN,NaN,0.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Subpolar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.46,0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Tropical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.19,NaN,0.36,0.26,0.16,NaN
6,WBC,0.53,NaN,NaN,NaN,NaN,NaN,0.34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
#Replace NaN with 0
df05_piv = df05_piv.replace(np.nan,0, regex=True)
df05_piv

MARCATS,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.35,0.00,...,0.00,0.00,0.00,0.59,0.00,0.0,0.00,0.00,0.00,0.00
1,Indian margins,0.00,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.2,0.00,0.00,0.00,0.31
2,Marginal sea,0.00,0.00,0.01,0.00,0.00,0.18,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
3,Polar,0.00,0.69,0.00,0.11,0.00,0.00,0.00,0.00,0.09,...,0.00,0.00,0.86,0.00,0.00,0.0,0.00,0.00,0.00,0.00
4,Subpolar,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.46,0.27,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
5,Tropical,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.19,0.0,0.36,0.26,0.16,0.00
6,WBC,0.53,0.00,0.00,0.00,0.00,0.00,0.34,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00


In [152]:
df05_piv.columns.name

'MARCATS'

In [153]:
df05_piv.columns.name = ''
df05_piv

,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.35,0.00,...,0.00,0.00,0.00,0.59,0.00,0.0,0.00,0.00,0.00,0.00
1,Indian margins,0.00,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.2,0.00,0.00,0.00,0.31
2,Marginal sea,0.00,0.00,0.01,0.00,0.00,0.18,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
3,Polar,0.00,0.69,0.00,0.11,0.00,0.00,0.00,0.00,0.09,...,0.00,0.00,0.86,0.00,0.00,0.0,0.00,0.00,0.00,0.00
4,Subpolar,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.46,0.27,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
5,Tropical,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.19,0.0,0.36,0.26,0.16,0.00
6,WBC,0.53,0.00,0.00,0.00,0.00,0.00,0.34,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00


In [154]:
#Save to tsv for stacked bar chart ( abs anthro flux in Tg/[m2 yr] )
df05_piv.to_csv('/homel/cnangini/PROJECTS/CO2airsea/orca05_absAnthro.tsv', float_format='%.2f', sep='\t')

In [155]:
#Create table for abs_anthro per Surface Area

#Pivot orig table
df05_piv_perArea = df05.pivot(columns='MARCATS', index='RCLASS', values='abs_anthro_perArea').reset_index()

#Replace NaN with 0
df05_piv = df05_piv.replace(np.nan,0, regex=True)
df05_piv_perArea

MARCATS,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4973617,NaN,...,NaN,NaN,NaN,2.124473,NaN,NaN,NaN,NaN,NaN,NaN
1,Indian margins,NaN,NaN,NaN,NaN,0.73359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.745549,NaN,NaN,NaN,0.3046401
2,Marginal sea,NaN,NaN,0.03894074,NaN,NaN,0.2822076,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Polar,NaN,20.40405,NaN,2.191623,NaN,NaN,NaN,NaN,1.118933,...,NaN,NaN,2.454693,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Subpolar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.222765,1.271076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Tropical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5140248,NaN,0.6586213,1.495641,0.08925441,NaN
6,WBC,1.204956,NaN,NaN,NaN,NaN,NaN,1.937559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
#Replace NaN with 0
df05_piv_perArea = df05_piv_perArea.replace(np.nan,0, regex=True)
df05_piv_perArea

MARCATS,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.497362,0.000000,...,0.000000,0.000000,0.000000,2.124473,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,Indian margins,0.000000,0.000000,0.000000,0.000000,0.73359,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.745549,0.000000,0.000000,0.000000,0.30464
2,Marginal sea,0.000000,0.000000,0.038941,0.000000,0.00000,0.282208,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,Polar,0.000000,20.404047,0.000000,2.191623,0.00000,0.000000,0.000000,0.000000,1.118933,...,0.000000,0.000000,2.454693,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,Subpolar,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,6.222765,1.271076,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
5,Tropical,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.514025,0.000000,0.658621,1.495641,0.089254,0.00000
6,WBC,1.204956,0.000000,0.000000,0.000000,0.00000,0.000000,1.937559,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [157]:
#Save to tsv for stacked bar chart ( abs anthro flux per Surface Area in Tg/yr )
df05_piv_perArea.to_csv('/homel/cnangini/PROJECTS/CO2airsea/orca05_absAnthroPerArea_TgPerYr.tsv', float_format='%.2f', sep='\t')

In [158]:
#============================
# PART II - SANKEY DIAGRAM
# Build json file
#============================

In [159]:
#Get sources list
sources = np.unique(df05.iloc[:,1].values) #RCLASS COL
sources

array([u'EBC', u'Indian margins', u'Marginal sea', u'Polar', u'Subpolar',
       u'Tropical', u'WBC'], dtype=object)

In [160]:
#Get regions list
regions = df05.iloc[:,0].values #MARCAT col
regions

array([u'Antarctic Shelves', u'North Eastern Atlantic',
       u'Northern Australia', u'Southern America', u'China Sea & Kuroshio',
       u'Florida Upwelling', u'South East Asia', u'Sea of Okhotsk',
       u'North Western Pacific', u'Southern Greenland',
       u'Barents & Kara Sea', u'North Eastern Pacific',
       u'Southern Western Africa', u'Brazilian Current', u'Gulf of Mexico',
       u'Tropical Eastern Indian', u'Northern Greenland', u'New Zealand',
       u'Mediterranean Sea', u'Tropical Western Atlantic', u'Sea of Japan',
       u'Southern Australia', u'Sea of Labrador', u'Agulhas Current',
       u'Eastern Arabian Sea', u'Carribean Sea', u'Canadian Archipeloagos',
       u'Siberian Shelves', u'Norwegian Basin', u'Hudson Bay',
       u'Iberian Upwelling', u'Bay of Bengal', u'Moroccan Upwelling',
       u'Tropical Eastern Pacific', u'Peruvian Upwelling Current',
       u'Leeuwin Current', u'Eastern Austrialian Current',
       u'Tropical Eastern Atlantic', u'California Current

In [161]:
nodes = sources.tolist() + regions.tolist()
nodes

[u'EBC',
 u'Indian margins',
 u'Marginal sea',
 u'Polar',
 u'Subpolar',
 u'Tropical',
 u'WBC',
 u'Antarctic Shelves',
 u'North Eastern Atlantic',
 u'Northern Australia',
 u'Southern America',
 u'China Sea & Kuroshio',
 u'Florida Upwelling',
 u'South East Asia',
 u'Sea of Okhotsk',
 u'North Western Pacific',
 u'Southern Greenland',
 u'Barents & Kara Sea',
 u'North Eastern Pacific',
 u'Southern Western Africa',
 u'Brazilian Current',
 u'Gulf of Mexico',
 u'Tropical Eastern Indian',
 u'Northern Greenland',
 u'New Zealand',
 u'Mediterranean Sea',
 u'Tropical Western Atlantic',
 u'Sea of Japan',
 u'Southern Australia',
 u'Sea of Labrador',
 u'Agulhas Current',
 u'Eastern Arabian Sea',
 u'Carribean Sea',
 u'Canadian Archipeloagos',
 u'Siberian Shelves',
 u'Norwegian Basin',
 u'Hudson Bay',
 u'Iberian Upwelling',
 u'Bay of Bengal',
 u'Moroccan Upwelling',
 u'Tropical Eastern Pacific',
 u'Peruvian Upwelling Current',
 u'Leeuwin Current',
 u'Eastern Austrialian Current',
 u'Tropical Eastern Atl

In [162]:
#df05.iloc['California Current']
df05.iloc[0]['abs_anthro']

0.69

In [163]:
df05.head(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro,total_perArea,anthro_perArea,abs_anthro_perArea
28,Antarctic Shelves,Polar,-71.76,2462,-0.904177,-0.69,0.69,-26.73749,-20.40405,20.40405
14,North Eastern Atlantic,Marginal sea,56.69,1020.02,-2.16314,-0.53,0.53,-26.50162,-6.493274,6.493274
40,Northern Australia,Tropical,-11.15,2278.28,-0.287668,-0.23,0.23,-7.871868,-6.293817,6.293817
30,Southern America,Subpolar,-47.92,1126.28,-1.5125,-0.46,0.46,-20.46072,-6.222765,6.222765
46,China Sea & Kuroshio,WBC,33.13,1131.67,-1.9898,-0.45,0.45,-27.04633,-6.11662,6.11662


In [164]:
file = open('sankey_orca05_anthroflux_perSurfaceArea.json', 'w')

file.write('{\n')
file.write('"nodes": [\n')
for node in nodes:
    file.write('{"name": "%s"},\n' %(node))
# remove last comma
file.seek(-2, os.SEEK_END)
file.truncate()
file.write('\n],\n')

file.write('"links": [\n')
idx = 0;
#for source in sources:
for region in regions:
    source = df05.iloc[idx]['RCLASS']
    #for region in regions:
    print idx
    print source
    print region
    print df05.iloc[idx]['abs_anthro_perArea']
    file.write('{"source": "%s", "target": "%s", "value": "%.2f"},\n' %(source, region, df05.iloc[idx]['abs_anthro_perArea']))
    idx = idx + 1
# remove last comma
file.seek(-2, os.SEEK_END)
file.truncate()
file.write('\n]\n')
file.write('}\n')

file.close()

0
Polar
Antarctic Shelves
20.40404658
1
Marginal sea
North Eastern Atlantic
6.4932739166
2
Tropical
Northern Australia
6.2938168484
3
Subpolar
Southern America
6.2227645768
4
WBC
China Sea & Kuroshio
6.1166197665
5
WBC
Florida Upwelling
5.278185906
6
Tropical
South East Asia
5.115773164
7
Marginal sea
Sea of Okhotsk
4.03365364956
8
Subpolar
North Western Pacific
2.9545042152
9
Polar
Southern Greenland
2.4546928744
10
Polar
Barents & Kara Sea
2.191623148
11
Subpolar
North Eastern Pacific
2.1448138788
12
EBC
Southern Western Africa
2.12447301008
13
WBC
Brazilian Current
1.93755902918
14
Marginal sea
Gulf of Mexico
1.80414060096
15
Indian margins
Tropical Eastern Indian
1.7455490212
16
Polar
Northern Greenland
1.69937921764
17
Subpolar
New Zealand
1.64241200552
18
Marginal sea
Mediterranean Sea
1.52455623
19
Tropical
Tropical Western Atlantic
1.49564070838
20
Marginal sea
Sea of Japan
1.42912931544
21
Subpolar
Southern Australia
1.27107560556
22
Subpolar
Sea of Labrador
1.25205991047
23
W